In [ ]:
from collections import namedtuple

from docplex.mp.model import Model
from docplex.util.environment import get_environment


# ----------------------------------------------------------------------------
# Initialize the problem data
# ----------------------------------------------------------------------------
nbs = (8, 3, 2)

team_div1 = {"Baltimore Ravens", "Cincinnati Bengals", "Cleveland Browns",
             "Pittsburgh Steelers", "Houston Texans", "Indianapolis Colts",
             "Jacksonville Jaguars", "Tennessee Titans", "Buffalo Bills",
             "Miami Dolphins", "New England Patriots", "New York Jets",
             "Denver Broncos", "Kansas City Chiefs", "Oakland Raiders",
             "San Diego Chargers"}

team_div2 = {"Chicago Bears", "Detroit Lions", "Green Bay Packers",
             "Minnesota Vikings", "Atlanta Falcons", "Carolina Panthers",
             "New Orleans Saints", "Tampa Bay Buccaneers", "Dallas Cowboys",
             "New York Giants", "Philadelphia Eagles", "Washington Redskins",
             "Arizona Cardinals", "San Francisco 49ers", "Seattle Seahawks",
             "St. Louis Rams"}

Match = namedtuple("Matches", ["team1", "team2", "is_divisional"])


# ----------------------------------------------------------------------------
# Build the model
# ----------------------------------------------------------------------------
def build_sports(**kwargs):
    print("* building sport scheduling model instance")
    mdl = Model('sportSchedCPLEX', **kwargs)

    nb_teams_in_division, nb_intra_divisional, nb_inter_divisional = nbs
    assert len(team_div1) == len(team_div2)
    mdl.teams = list(team_div1 | team_div2)
    # team index ranges from 1 to 2N
    team_range = range(1, 2 * nb_teams_in_division + 1)

    # Calculate the number of weeks necessary.
    nb_weeks = (nb_teams_in_division - 1) * nb_intra_divisional + nb_teams_in_division * nb_inter_divisional
    weeks = range(1, nb_weeks + 1)
    mdl.weeks = weeks

    print("{0} games, {1} intradivisional, {2} interdivisional"
          .format(nb_weeks, (nb_teams_in_division - 1) * nb_intra_divisional,
                  nb_teams_in_division * nb_inter_divisional))

    # Season is split into two halves.
    first_half_weeks = range(1, nb_weeks // 2 + 1)
    nb_first_half_games = nb_weeks // 3

    # All possible matches (pairings) and whether of not each is intradivisional.
    matches = [Match(t1, t2, 1 if (t2 <= nb_teams_in_division or t1 > nb_teams_in_division) else 0)
               for t1 in team_range for t2 in team_range if t1 < t2]
    mdl.matches = matches
    # Number of games to play between pairs depends on
    # whether the pairing is intradivisional or not.
    nb_play = {m: nb_intra_divisional if m.is_divisional == 1 else nb_inter_divisional for m in matches}

    plays = mdl.binary_var_matrix(keys1=matches, keys2=weeks,
                                  name=lambda mw: "play_%d_%d_w%d" % (mw[0].team1, mw[0].team2, mw[1]))
    mdl.plays = plays

    for m in matches:
        mdl.add_constraint(mdl.sum(plays[m, w] for w in weeks) == nb_play[m],
                           "correct_nb_games_%d_%d" % (m.team1, m.team2))

    for w in weeks:
        # Each team must play exactly once in a week.
        for t in team_range:
            max_teams_in_division = (plays[m, w] for m in matches if m.team1 == t or m.team2 == t)
            mdl.add_constraint(mdl.sum(max_teams_in_division) == 1,
                               "plays_exactly_once_%d_%s" % (w, t))

    # Games between the same teams cannot be on successive weeks.
    mdl.add_constraints(plays[m, w] + plays[m, w + 1] <= 1
                        for w in weeks for m in matches if w < nb_weeks)

    # Some intradivisional games should be in the first half.
    for t in team_range:
        max_teams_in_division = [plays[m, w] for w in first_half_weeks for m in matches if
                                 m.is_divisional == 1 and (m.team1 == t or m.team2 == t)]

        mdl.add_constraint(mdl.sum(max_teams_in_division) >= nb_first_half_games,
                           "in_division_first_half_%s" % t)

    # postpone divisional matches as much as possible
    # we weight each play variable with the square of w.
    mdl.maximize(mdl.sum(plays[m, w] * w * w for w in weeks for m in matches if m.is_divisional))
    return mdl

# a named tuple to store solution
TSolution = namedtuple("TSolution", ["week", "is_divisional", "team1", "team2"])


def print_sports_solution(mdl):
    # iterate with weeks first
    solution = [TSolution(w, m.is_divisional, mdl.teams[m.team1], mdl.teams[m.team2])
                for w in mdl.weeks for m in mdl.matches
                if mdl.plays[m, w].to_bool()]

    currweek = 0
    print("Intradivisional games are marked with a *")
    for s in solution:
        # assume records are sorted by increasing week indices.
        if s.week != currweek:
            currweek = s.week
            print(" == == == == == == == == == == == == == == == == ")
            print("On week %d" % currweek)

        print("    {0:s}{1} will meet the {2}".format("*" if s.is_divisional else "", s.team1, s.team2))


# ----------------------------------------------------------------------------
# Solve the model and display the result
# ----------------------------------------------------------------------------
if __name__ == '__main__':
    # Build the model
    model = build_sports()
    model.print_information()
    # Solve the model. If a key has been specified above, the solve
    # will use IBM Decision Optimization on cloud.
    if model.solve():
        model.report()
        print_sports_solution(model)
        # Save the CPLEX solution as "solution.json" program output
        with get_environment().get_output_stream("solution.json") as fp:
            model.solution.export(fp, "json")
    else:
        print("Problem could not be solved: " + model.solve_details.get_status())
    model.end()

In [6]:
import itertools

teams = ["Team1", "Team2", "Team3", "Team4"]  # List of 4 NFL teams for illustration
opponents = {
    "Team1": ["Team2", "Team3", "Team4"],
    "Team2": ["Team1", "Team3", "Team4"],
    "Team3": ["Team1", "Team2", "Team4"],
    "Team4": ["Team1", "Team2", "Team3"],
}

# Initialize an empty schedule
schedule = {}

# Generate the schedule
for team in teams:
    opponents_to_play = opponents[team].copy()
    for i in range(len(opponents_to_play)):
        # Choose an opponent for the current game
        opponent = opponents_to_play.pop(0)

        # Check if this matchup already exists in the schedule
        if (team, opponent) in schedule.values():
            continue
        else:
            schedule[team + " vs " + opponent] = (team, opponent)


            
        
        if any(opponent in match for match in schedule.values()):
            opponents_to_play.append(opponent)
            opponent = opponents_to_play.pop(0)
        # Assign the matchup to the schedule

# Print the schedule
for matchup, teams in schedule.items():
    print(f"{matchup}: {teams[0]} vs {teams[1]}")

schedule

Team1 vs Team2: Team1 vs Team2
Team1 vs Team3: Team1 vs Team3
Team1 vs Team4: Team1 vs Team4
Team2 vs Team3: Team2 vs Team3
Team2 vs Team1: Team2 vs Team1
Team2 vs Team4: Team2 vs Team4
Team3 vs Team2: Team3 vs Team2
Team3 vs Team1: Team3 vs Team1
Team3 vs Team4: Team3 vs Team4
Team4 vs Team2: Team4 vs Team2
Team4 vs Team1: Team4 vs Team1
Team4 vs Team3: Team4 vs Team3


{'Team1 vs Team2': ('Team1', 'Team2'),
 'Team1 vs Team3': ('Team1', 'Team3'),
 'Team1 vs Team4': ('Team1', 'Team4'),
 'Team2 vs Team3': ('Team2', 'Team3'),
 'Team2 vs Team1': ('Team2', 'Team1'),
 'Team2 vs Team4': ('Team2', 'Team4'),
 'Team3 vs Team2': ('Team3', 'Team2'),
 'Team3 vs Team1': ('Team3', 'Team1'),
 'Team3 vs Team4': ('Team3', 'Team4'),
 'Team4 vs Team2': ('Team4', 'Team2'),
 'Team4 vs Team1': ('Team4', 'Team1'),
 'Team4 vs Team3': ('Team4', 'Team3')}

In [ ]:
teams_dict = {
    "Arizona Cardinals": ["NFC West", "NFC"],
    "Atlanta Falcons": ["NFC South", "NFC"],
    "Baltimore Ravens": ["AFC North", "AFC"],
    "Buffalo Bills": ["AFC East", "AFC"],
    "Carolina Panthers": ["NFC South", "NFC"],
    "Chicago Bears": ["NFC North", "NFC"],
    "Cincinnati Bengals": ["AFC North", "AFC"],
    "Cleveland Browns": ["AFC North", "AFC"],
    "Dallas Cowboys": ["NFC East", "NFC"],
    "Denver Broncos": ["AFC West", "AFC"],
    "Detroit Lions": ["NFC North", "NFC"],
    "Green Bay Packers": ["NFC North", "NFC"],
    "Houston Texans": ["AFC South", "AFC"],
    "Indianapolis Colts": ["AFC South", "AFC"],
    "Jacksonville Jaguars": ["AFC South", "AFC"],
    "Kansas City Chiefs": ["AFC West", "AFC"],
    "Las Vegas Raiders": ["AFC West", "AFC"],
    "Los Angeles Chargers": ["AFC West", "AFC"],
    "Los Angeles Rams": ["NFC West", "NFC"],
    "Miami Dolphins": ["AFC East", "AFC"],
    "Minnesota Vikings": ["NFC North", "NFC"],
    "New England Patriots": ["AFC East", "AFC"],
    "New Orleans Saints": ["NFC South", "NFC"],
    "New York Giants": ["NFC East", "NFC"],
    "New York Jets": ["AFC East", "AFC"],
    "Philadelphia Eagles": ["NFC East", "NFC"],
    "Pittsburgh Steelers": ["AFC North", "AFC"],
    "San Francisco 49ers": ["NFC West", "NFC"],
    "Seattle Seahawks": ["NFC West", "NFC"],
    "Tampa Bay Buccaneers": ["NFC South", "NFC"],
    "Tennessee Titans": ["AFC South", "AFC"],
    "Washington Football Team": ["NFC East", "NFC"]
}

In [ ]:
NFC = [team for team in teams_dict.keys() if "NFC" in teams_dict[team]]
AFC = [team for team in teams_dict.keys() if "AFC" in teams_dict[team]]

A_North = [team for team in teams_dict.keys() if "AFC North" in teams_dict[team]]
A_South = [team for team in teams_dict.keys() if "AFC South" in teams_dict[team]]
A_East = [team for team in teams_dict.keys() if "AFC East" in teams_dict[team]]
A_West = [team for team in teams_dict.keys() if "AFC West" in teams_dict[team]]

N_North = [team for team in teams_dict.keys() if "NFC North" in teams_dict[team]]
N_South = [team for team in teams_dict.keys() if "NFC South" in teams_dict[team]]
N_East = [team for team in teams_dict.keys() if "NFC East" in teams_dict[team]]
N_West = [team for team in teams_dict.keys() if "NFC West" in teams_dict[team]]

AFC_tiers = (A_North, A_South, A_East, A_West)
NFC_tiers = (N_North, N_South, N_East, N_West)

NFL_tiers = (AFC_tiers, NFC_tiers)

In [ ]:
# first try at the round robin matching algo
import random, copy, json

full_schedule = {}

# for week_num in range(16):
for week_num in range(2):
    afccopy = copy.copy(AFC)
    nfccopy = copy.copy(NFC)
    random.shuffle(afccopy)

    weekly_matchups = [{"home": afc_team.name, "away": nfc_team.name} for afc_team, nfc_team in zip(afccopy, nfccopy)]


    full_schedule[f"week_{week_num+1}"] = weekly_matchups

full_schedule_json = json.dumps(full_schedule, indent=4)  # indent for pretty printing
# print(full_schedule_json)


In [ ]:
# Generate a list of bye weeks, assign to teams
# Byes can happen between week 4 and week 14
import copy

# refresh the team objects
# create_new_league()

# print a list of teams with no bye week yet for debugging
no_bye_week = [team.name for team in nfl_teams if team.bye_week == None]
print(no_bye_week)

eligible_weeks = list(range(4, 15))

for team in nfl_teams:
    week_to_add = random.choice(eligible_weeks)
    team.set_bye_week(week_to_add)
    # print(team.bye_week, team.name)

# print a list of teams with no bye week for debugging. this should now an empty blank now
no_bye_week_after = [team.name for team in nfl_teams if team.bye_week == None]
print(no_bye_week_after)


['New England Patriots', 'Buffalo Bills', 'Miami Dolphins', 'New York Jets', 'Baltimore Ravens', 'Pittsburgh Steelers', 'Cleveland Browns', 'Cincinnati Bengals', 'Houston Texans', 'Indianapolis Colts', 'Tennessee Titans', 'Jacksonville Jaguars', 'Kansas City Chiefs', 'Denver Broncos', 'Las Vegas Raiders', 'Los Angeles Chargers', 'Dallas Cowboys', 'Washington Football Team', 'Philadelphia Eagles', 'New York Giants', 'Green Bay Packers', 'Chicago Bears', 'Minnesota Vikings', 'Detroit Lions', 'Tampa Bay Buccaneers', 'New Orleans Saints', 'Carolina Panthers', 'Atlanta Falcons', 'Seattle Seahawks', 'Los Angeles Rams', 'Arizona Cardinals', 'San Francisco 49ers']
[]


In [ ]:
# def generate_bye_weeks(list_of_teams):    
#     eligible_weeks = list(range(5, 15))
# #     in the NFLTeam objects, set the bye week
#     for team in list_of_teams:
#         week_to_add = random.choice(eligible_weeks)
#         team.set_bye_week(week_to_add)
#     return {week: [team.name for team in nfl_teams if team.bye_week == week] for week in range(4, 15)}

# def has_empty_bye_weeks(dictionary):
#     """ checks if there are any weeks where no team has a bye
#     Args:
#         dictionary (dictionary): a dictionary with key = week_num and value = a list of teams with a bye in that week (created)
#     Returns:
#         bool: returns True if generate_bye_weeks() creates a bye week schedule where any week between 4 and 14 has no teams with a bye.
#     """
#     for value in dictionary.values():
#         if isinstance(value, list) and not value:
#             return True
#     return False


In [ ]:
    def show_schedule(self):
        for matchup in self.schedule:
            if matchup[home_indicator] == True :
                print(self.name, "vs.", matchup.opponent)
            else:
                print(self.name, "@", matchup.opponent)

In [14]:
import random
sorted(random.sample(range(18), k = 6))

[5, 10, 11, 13, 14, 17]

In [18]:
None == None == None

True

In [5]:
# Step 1: Create an empty dictionary
my_dict = {}

# Step 2: Initialize the value associated with a key as an empty list
my_key = 1
my_dict[my_key] = {"home": "a", "away": "B"}

print(my_dict)

# Step 3: Append a value to the list associated with the key
new_value = 'new_item'
my_dict[my_key].append(new_value)

# You can continue appending more values to the list
my_dict[my_key].append('another_item')

print(my_dict)  # Output: {'example_key': ['new_item', 'another_item']}


{1: {'home': 'a', 'away': 'B'}}


AttributeError: 'dict' object has no attribute 'append'

In [3]:
debug = True
import random

eligible_weeks = list(range(4, 15)) 

weekly_bye_count_list = [random.choice([2,4]) for week in eligible_weeks]
while sum(weekly_bye_count_list) != 32:
    weekly_bye_count_list =  [random.choice([2,4]) for week in eligible_weeks]

if debug:
    print(weekly_bye_count_list)

# add the weeks at start and end of season to fill out the full schedule
weekly_bye_count_list = [0,0,0] + weekly_bye_count_list + [0,0,0,0]

if debug:
    print(weekly_bye_count_list)

# list of how many teams from each conference are on a bye for in each week (index)
weekly_bye_slots_per_conference = [value//2 for value in weekly_bye_count_list]
if debug:
    print(weekly_bye_slots_per_conference)


[4, 2, 2, 2, 2, 4, 2, 4, 2, 4, 4]
[0, 0, 0, 4, 2, 2, 2, 2, 4, 2, 4, 2, 4, 4, 0, 0, 0, 0]
[0, 0, 0, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 2, 0, 0, 0, 0]


In [ ]:
game_type_outline = {"in_conf":}